In [8]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
try:
    import tensorflow.compat.v2 as tf
except Exception:
    pass

##tf.enable_v2_behaborior()

from tensorflow.compat.v2 import keras
from tensorflow.compat.v2.keras import layers

print(tf.__version__)

1.14.0


In [5]:
train_data = pd.read_csv('datasets/train.csv')
test_data = pd.read_csv('datasets/test.csv')
data_all = pd.concat([train_data, test_data], ignore_index=True, sort=False)

In [6]:
print(train_data.shape, test_data.shape, data_all.shape)

(116058, 29) (35866, 28) (151924, 29)


In [20]:
data_all.head(10)

,interest_rate,unpaid_principal_bal,loan_term,origination_date,first_payment_date,loan_to_value,number_of_borrowers,debt_to_income_ratio,borrower_credit_score,loan_purpose,insurance_percent,co-borrower_credit_score,insurance_type,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,m13
0,4.250,214000,360,2012-03-01,05/2012,95,1.0,22.0,694.0,C86,30.0,0.0,0.0,0,0,0,0,0,0,1,0,0,0,0,0,1.0
1,4.875,144000,360,2012-01-01,03/2012,72,1.0,44.0,697.0,B12,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,1,0,1.0
2,3.250,366000,180,2012-01-01,03/2012,49,1.0,33.0,780.0,B12,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1.0
3,4.750,135000,360,2012-02-01,04/2012,46,2.0,44.0,633.0,B12,0.0,638.0,0.0,0,0,0,0,0,0,0,0,1,1,1,1,1.0
4,4.750,124000,360,2012-02-01,04/2012,80,1.0,43.0,681.0,C86,0.0,0.0,0.0,0,1,2,3,4,5,6,7,8,9,10,11,1.0
5,4.375,150000,360,2012-02-01,04/2012,80,1.0,46.0,675.0,C86,0.0,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,1.0
6,4.000,59000,360,2012-02-01,04/2012,95,1.0,44.0,723.0,C86,30.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1.0
7,4.500,319000,300,2012-01-01,03/2012,62,1.0,45.0,652.0,A23,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,1.0
8,4.000,520000,360,2012-03-01,05/2012,76,1.0,35.0,808.0,C86,0.0,0.0,0.0,0,0,0,0,1,0,1,0,1,2,0,1,1.0
9,4.000,214000,360,2012-01-01,03/2012,95,2.0,41.0,702.0,A23,30.0,700.0,0.0,0,0,0,0,0,1,1,1,1,1,2,2,1.0


In [22]:
data_all.tail()

,interest_rate,unpaid_principal_bal,loan_term,origination_date,first_payment_date,loan_to_value,number_of_borrowers,debt_to_income_ratio,borrower_credit_score,loan_purpose,insurance_percent,co-borrower_credit_score,insurance_type,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,m13
151919,4.125,232000,240,01/01/12,Apr-12,80,2.0,30.0,684.0,C86,0.0,712.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
151920,3.375,204000,180,01/01/12,Mar-12,80,1.0,30.0,812.0,B12,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
151921,4.250,200000,360,01/02/12,Apr-12,52,2.0,38.0,624.0,B12,0.0,646.0,0.0,0,0,0,0,0,0,0,0,1,0,0,0,NaN
151922,4.375,400000,360,01/02/12,Apr-12,66,1.0,34.0,753.0,A23,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
151923,4.375,182000,360,01/02/12,Apr-12,70,1.0,3.0,0.0,C86,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,NaN


In [23]:
data_all['total_score'] = data_all['borrower_credit_score'] + data_all['co-borrower_credit_score']
data_all.head()

,interest_rate,unpaid_principal_bal,loan_term,origination_date,first_payment_date,loan_to_value,number_of_borrowers,debt_to_income_ratio,borrower_credit_score,loan_purpose,insurance_percent,co-borrower_credit_score,insurance_type,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,m13,total_score
0,4.250,214000,360,2012-03-01,05/2012,95,1.0,22.0,694.0,C86,30.0,0.0,0.0,0,0,0,0,0,0,1,0,0,0,0,0,1.0,694.0
1,4.875,144000,360,2012-01-01,03/2012,72,1.0,44.0,697.0,B12,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,1,0,1.0,697.0
2,3.250,366000,180,2012-01-01,03/2012,49,1.0,33.0,780.0,B12,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,780.0
3,4.750,135000,360,2012-02-01,04/2012,46,2.0,44.0,633.0,B12,0.0,638.0,0.0,0,0,0,0,0,0,0,0,1,1,1,1,1.0,1271.0
4,4.750,124000,360,2012-02-01,04/2012,80,1.0,43.0,681.0,C86,0.0,0.0,0.0,0,1,2,3,4,5,6,7,8,9,10,11,1.0,681.0


In [24]:
data_all['insurance_type'].value_counts()

0.0    151432
1.0       492
Name: insurance_type, dtype: int64

In [25]:
data_all['prem_paid_bro'] = data_all.loc[data_all['insurance_type']==0.0, 'prem_paid_bro'] = 0
data_all['prem_paid_lend'] = data_all.loc[data_all['insurance_type']==1.0, 'prem_paid_bro'] = 1

In [26]:
data_all.tail()

,interest_rate,unpaid_principal_bal,loan_term,origination_date,first_payment_date,loan_to_value,number_of_borrowers,debt_to_income_ratio,borrower_credit_score,loan_purpose,insurance_percent,co-borrower_credit_score,insurance_type,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,m13,total_score,prem_paid_bro,prem_paid_lend
151919,4.125,232000,240,01/01/12,Apr-12,80,2.0,30.0,684.0,C86,0.0,712.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,1396.0,0,1
151920,3.375,204000,180,01/01/12,Mar-12,80,1.0,30.0,812.0,B12,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,812.0,0,1
151921,4.250,200000,360,01/02/12,Apr-12,52,2.0,38.0,624.0,B12,0.0,646.0,0.0,0,0,0,0,0,0,0,0,1,0,0,0,NaN,1270.0,0,1
151922,4.375,400000,360,01/02/12,Apr-12,66,1.0,34.0,753.0,A23,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,753.0,0,1
151923,4.375,182000,360,01/02/12,Apr-12,70,1.0,3.0,0.0,C86,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0.0,0,1


In [29]:
data_stats = data_all.describe()
#data_stats.drop(['m13'], axis=1, inplace=True)
data_stats=data_stats.transpose()
data_stats

,count,mean,std,min,25%,50%,75%,max
interest_rate,151924.0,3.869879,0.460908,2.25,3.5,3.875,4.125,6.75
unpaid_principal_bal,151924.0,208117.302072,114655.780507,11000.00,120000.0,182000.000,278000.000,1200000.00
loan_term,151924.0,292.481458,89.653613,60.00,180.0,360.000,360.000,360.00
loan_to_value,151924.0,67.421645,17.281065,5.00,57.0,72.000,80.000,97.00
number_of_borrowers,151924.0,1.594040,0.491078,1.00,1.0,2.000,2.000,2.00
debt_to_income_ratio,151924.0,30.747150,9.729672,1.00,23.0,31.000,39.000,64.00
borrower_credit_score,151924.0,769.926713,42.109207,0.00,751.0,782.000,799.000,840.00
insurance_percent,151924.0,2.772860,8.080634,0.00,0.0,0.000,0.000,40.00
co-borrower_credit_score,151924.0,460.278514,381.798443,0.00,0.0,741.000,791.000,836.00
insurance_type,151924.0,0.003238,0.056815,0.00,0.0,0.000,0.000,1.00


In [32]:
data_all.drop(['loan_purpose', 'origination_date', 'first_payment_date'], axis=1, inplace=True)

In [33]:
train_datasets=data_all.iloc[:116058, :]
test_datasets=data_all.iloc[116058:, :]

In [34]:
train_datasets.head()

,interest_rate,unpaid_principal_bal,loan_term,loan_to_value,number_of_borrowers,debt_to_income_ratio,borrower_credit_score,insurance_percent,co-borrower_credit_score,insurance_type,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,m13,total_score,prem_paid_bro,prem_paid_lend
0,4.250,214000,360,95,1.0,22.0,694.0,30.0,0.0,0.0,0,0,0,0,0,0,1,0,0,0,0,0,1.0,694.0,0,1
1,4.875,144000,360,72,1.0,44.0,697.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,1,0,1.0,697.0,0,1
2,3.250,366000,180,49,1.0,33.0,780.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,780.0,0,1
3,4.750,135000,360,46,2.0,44.0,633.0,0.0,638.0,0.0,0,0,0,0,0,0,0,0,1,1,1,1,1.0,1271.0,0,1
4,4.750,124000,360,80,1.0,43.0,681.0,0.0,0.0,0.0,0,1,2,3,4,5,6,7,8,9,10,11,1.0,681.0,0,1


In [35]:
test_datasets.head()

,interest_rate,unpaid_principal_bal,loan_term,loan_to_value,number_of_borrowers,debt_to_income_ratio,borrower_credit_score,insurance_percent,co-borrower_credit_score,insurance_type,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,m13,total_score,prem_paid_bro,prem_paid_lend
116058,3.875,417000,360,75,1.0,20.0,790.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,790.0,0,1
116059,4.500,113000,360,80,2.0,33.0,793.0,0.0,784.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,1577.0,0,1
116060,4.500,72000,360,75,1.0,34.0,710.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,710.0,0,1
116061,4.125,123000,180,41,2.0,24.0,798.0,0.0,813.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,1611.0,0,1
116062,3.250,166000,180,53,2.0,12.0,767.0,0.0,768.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,1535.0,0,1


In [55]:
train_datasets_labels=train_datasets.drop(['m13'], axis=1, inplace = True)
test_datasets_labels =test_datasets
test_datasets_labels.head()

KeyError: "['m13'] not found in axis"

In [37]:
def norm(x):
    return (x-data_stats['mean'])/data_stats['std']
normed_train_datasets=norm(train_datasets)
normed_test_datasets=norm(test_datasets)

In [49]:
def build_model():
    model = keras.Sequential([
        layers.Dense(64, activation=tf.nn.relu, input_shape=[len(train_datasets.keys())]),
        layers.Dense(64, activation=tf.nn.relu),
        layers.Dense(1)
    ])
    optimizer = tf.keras.optimizers.RMSprop(0.001)
    model.compile(loss="mean_squared_error",
                 optimizer=optimizer,                 
                 metrics = ["mean_absolute_error", "mean_squared_error"])
    return model

model= build_model()

In [50]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 64)                1728      
_________________________________________________________________
dense_25 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_26 (Dense)             (None, 1)                 65        
Total params: 5,953
Trainable params: 5,953
Non-trainable params: 0
_________________________________________________________________


In [45]:
example_batch=normed_train_datasets[:10]
example_result=model.predict(example_batch)
example_result

array([[nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan]], dtype=float32)

In [ ]:
class PrintDot(keras.callbacks.callback):
    def on_epoch_end(self.epoch, logs):
        if epoch % 100 == 0: print(' ')
        print('.', end=' ')